In [1]:
import numpy as np
import pprint
from operator import itemgetter

In [2]:
class MDP:
    def __init__(self, T, S, R, A, act_list):        
        # State space
        # Integer number of states
        self.S = S
        
        # Transition probabilities
        # Form: np ndarray of shape (start_state, action, end_state)
        self.T = T
        
        # Reward space
        # Form: vector, rewards for each state
        self.R = R
        
        # Action space
        # integer, number of possible actions
        self.A = A
        
        # Possible actions in the MDP
        self.actions = act_list
    

In [3]:
class Grid_world(MDP):
    def __init__(self, grid_size, reward_pos):
        S = grid_size*grid_size
        
        R = np.zeros((grid_size, grid_size))
        
        # Each row of reward_pos is a tuple: x, y, reward
        for row in reward_pos:
            R[row[0], row[1]] = row[2]
        R = R.flatten()
        
        A = 4
        act_list = ['S', 'E', 'N', 'W']
        
        T = np.zeros((S, A, S))
        for start_state in range(S):
            state_i = start_state/grid_size
            state_j = (start_state)%grid_size
            
            # Actions indexed as: 0:S, 1:E, 2:N, 3:W
            for act in range(A):
                feas_grid = np.zeros((grid_size, grid_size))
                if(act == 0 ):
                    if(state_i+1 < grid_size):                        
                        feas_grid[state_i+1, state_j] = 1
                    else:
                        feas_grid[state_i, state_j] = 1
                        
                elif(act == 1):
                    if(state_j+1 < grid_size):                        
                        feas_grid[state_i, state_j+1] = 1
                    else:
                        feas_grid[state_i, state_j] = 1                    
                    
                elif(act == 2):
                    if(state_i-1 >= 0):                        
                        feas_grid[state_i-1, state_j] = 1
                    else:
                        feas_grid[state_i, state_j] = 1                    
                    
                elif(act == 3):
                    if(state_j-1 >= 0):                        
                        feas_grid[state_i, state_j-1] = 1
                    else:
                        feas_grid[state_i, state_j] = 1                    
                    
                    
                # Flatten the feasibility grid and assign to transition matrix
                T[start_state, act, :] = feas_grid.flatten()
        MDP.__init__(self, T, S, R, A, act_list)

In [32]:
test_rewards = [[i, j, -1] for i in range(5) for j in range(5)]
test_rewards[2] = [0, 2, 1]
test_rewards[23] = [4,3, 1]
# test_rewards = [[0, 3, 5],
#                 [0, 1, 10]]
print test_rewards
gw = Grid_world(5, test_rewards)

[[0, 0, -1], [0, 1, -1], [0, 2, 1], [0, 3, -1], [0, 4, -1], [1, 0, -1], [1, 1, -1], [1, 2, -1], [1, 3, -1], [1, 4, -1], [2, 0, -1], [2, 1, -1], [2, 2, -1], [2, 3, -1], [2, 4, -1], [3, 0, -1], [3, 1, -1], [3, 2, -1], [3, 3, -1], [3, 4, -1], [4, 0, -1], [4, 1, -1], [4, 2, -1], [4, 3, 1], [4, 4, -1]]


In [33]:
print gw.T[0, 1, :]

[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


In [49]:
def sarsa(mdp, alpha = 0.1, gamma = 0.9):
    # Initialization
    Q = [[0 for i in range(mdp.A)] for j in range(mdp.S)]
    pol = [1]*mdp.S
    old_Q = Q
    n_iter = 0
    epsilon = 0.1
#     r = 0 # TODO: Remove once found better termination criteria
    while n_iter < 10000000: # TODO: Pick a finish condition for episode
        s = 0 # Initialize s, starting state
        
        # With prob epsilon, pick a random action
        if np.random.random_sample() <= epsilon:
            a = np.random.random_integers(0, mdp.A-1)         
        else:
            a = np.argmax(Q[s][:])
        r = 0
        while r!= 1: # TODO: Finish episode/trajectory on terminal state
            # Observe S and R
            s_new = np.argmax(mdp.T[s, a, :]) # TODO: Change to stochastic ?
            r = mdp.R[s_new]
            T_new = np.zeros((mdp.S, mdp.S))
            
            # Pick new action A' from S'
            if np.random.random_sample() <= epsilon:
                a_new = np.random.random_integers(0, mdp.A-1)         
            else:
                a_new = np.argmax(Q[s_new][:])
            Q[s][a] = Q[s][a] + alpha*(r + gamma*Q[s_new][a_new] - Q[s][a])
            s = s_new
            a = a_new
        
            n_iter += 1
    return Q

In [50]:
Q = sarsa(gw)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:26: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:14: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead


In [51]:
print np.reshape(np.argmax(Q, 1), (5,5))

[[1 1 0 3 3]
 [2 2 2 3 3]
 [1 1 2 0 0]
 [0 0 0 0 3]
 [1 1 1 0 3]]


In [40]:
print np.reshape(gw.R, (5,5))

[[-1. -1.  1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1.  1. -1.]]
